In [4]:
import os

def create_tree(directory, indent=""):
    print(indent + os.path.basename(directory) + "/")
    
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            create_tree(item_path, indent + "  ")
        else:
            print(indent + "  " + item)

# Replace 'your_directory_path' with the path of the directory you want to create a tree for.
directory_path = '/home/liranc6/ecg/ecg_forecasting/data/icentia11k-continuous-ecg/'

if os.path.exists(directory_path):
    create_tree(directory_path)
else:
    print(f"The directory '{directory_path}' does not exist.")

/
  ANNOTATORS
  index.html
  LICENSE.txt
  p00/
    index.html
    p00000/
      index.html
      RECORDS
      p00000_s00.atr
      p00000_s00.dat
      p00000_s00.hea
      p00000_s01.atr
      p00000_s01.dat
      p00000_s01.hea
      p00000_s02.atr
      p00000_s02.dat
      p00000_s02.hea
      p00000_s03.atr
      p00000_s03.dat
      p00000_s03.hea
      p00000_s04.atr
      p00000_s04.dat
      p00000_s04.hea
      p00000_s05.atr
      p00000_s05.dat
      p00000_s05.hea
      p00000_s06.atr
      p00000_s06.dat
      p00000_s06.hea
      p00000_s07.atr
      p00000_s07.dat
      p00000_s07.hea
      p00000_s08.atr
      p00000_s08.dat
      p00000_s08.hea
      p00000_s09.atr
      p00000_s09.dat
      p00000_s09.hea
      p00000_s10.atr
      p00000_s10.dat
      p00000_s10.hea
      p00000_s11.atr
      p00000_s11.dat
      p00000_s11.hea
      p00000_s12.atr
      p00000_s12.dat
      p00000_s12.hea
      p00000_s13.atr
      p00000_s13.dat
      p00000_s13.hea
      p0000

In [3]:
import numpy as np

# assuming signals and indices are defined
signals = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
indices = np.array([1, 3, 5, 7])

beats = np.zeros(signals.shape[0])
beats[indices] = 1

# stack signals and beats into a 2D array
result = np.vstack((signals, beats))

result

array([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0., 0.]])

In [5]:
result = np.concatenate((signals, beats), axis=1)
result

AxisError: axis 1 is out of bounds for array of dimension 1

In [1]:
import h5py
import numpy as np

def open_h5_file_and_show_first_sample(file_path):
    with h5py.File(file_path, 'r') as h5_file:
        keys = sorted(h5_file.keys())
        group_keys = sorted(h5_file.keys())
        print(f"Keys: {keys}")
        print(f"Group keys: {group_keys}")
        
        for key in keys:
            print(f"Key: {key}")
            print(f"Shape: {h5_file[key].shape}")
            print(f"Value: {h5_file[key][0]}")
            break
    
    


# file_path = "/mnt/qnap/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes_window.h5"
# file_path = "/mnt/qnap/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays.h5"
file_path = '/mnt/qnap/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes_window_temp.h5'
open_h5_file_and_show_first_sample(file_path)

Keys: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008']
Group keys: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008']
Key: 00000
Shape: (0,)


IndexError: Index (0) out of range for empty dimension

In [7]:
import os
import sys
sys.path.append("/home/liranc6/ecg/ecg_forecasting/liran_project/utils")
import data_preparation
# data_preparation.print_h5_hierarchy(file_path)
# print first dataset
def print_dataset(file_path):
    with h5py.File(file_path, 'r') as h5_file:
        keys = sorted(h5_file.keys())
        for key in keys:
            print(f"Key: {key}")
            print(f"Shape: {h5_file[key].shape}")
            print(f"Value: {h5_file[key][0]}")
            break

# print_dataset(file_path)
data_preparation.count_items(file_path)

Dataset: /00000, Count: 0
Dataset: /00001, Count: 0
Dataset: /00002, Count: 0
Dataset: /00003, Count: 0
Dataset: /00004, Count: 0
Dataset: /00005, Count: 0
Dataset: /00006, Count: 0
Dataset: /00007, Count: 0
Dataset: /00008, Count: 0
Total count: 0


In [12]:
import numpy as np
#create array with shape [1, 2, 10]
a = np.random.rand(2, 10)
print(f"{a=}")

window_size = 5
num_windows = a.shape[1] // window_size
print(f"{num_windows=}")
# Split the array into smaller arrays of window_size along the second axis
result = [a[:, i:i + window_size] for i in range(0, a.shape[1], window_size)]

# Convert the result to an array of subarrays
result = np.array(result)

# Print the result
print(result)

a=array([[0.75561255, 0.3418389 , 0.32594068, 0.64460466, 0.597923  ,
        0.87878053, 0.24768316, 0.53570814, 0.30807981, 0.16774639],
       [0.79225451, 0.38278559, 0.76761923, 0.5211686 , 0.80412334,
        0.64473882, 0.33466251, 0.44096894, 0.22946018, 0.88897458]])
num_windows=2
[[[0.75561255 0.3418389  0.32594068 0.64460466 0.597923  ]
  [0.79225451 0.38278559 0.76761923 0.5211686  0.80412334]]

 [[0.87878053 0.24768316 0.53570814 0.30807981 0.16774639]
  [0.64473882 0.33466251 0.44096894 0.22946018 0.88897458]]]


In [2]:
def mse_distance_batch(y_batch, y_pred_batch):
    
    total_error = F.mse_loss(y_batch.float(), y_pred_batch.float()).item()

    return total_error

In [4]:
import torch
import torch.nn.functional as F

# Example data
pred = torch.tensor([2.5, 0.0, 2.0, 8.0])
target = torch.tensor([3.0, -0.5, 2.0, 7.0])

# Calculate MSE
def mse(pred, target):
    return F.mse_loss(pred, target)

# Calculate RMSE
def rmse(pred, target):
    return torch.sqrt(mse(pred, target))

# Calculate MAE
def mae(pred, target):
    return F.l1_loss(pred, target)

mse, rmse, mae = mse(pred, target), rmse(pred, target), mae(pred, target)
print(f"MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}")

mse_distance_batch(pred, target)

MSE: 0.38, RMSE: 0.61, MAE: 0.50


0.375

In [10]:
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

a = [[1, 4, 3, 2, 4, 2, 4], [1, 4, 3, 2, 4]]
b = [[5, 3, 4, 6, 4, 5, 7], [5, 3, 4, 6, 4]]

# Convert lists to tensors and pad
a = pad_sequence([torch.tensor(x) for x in a], batch_first=True)
b = pad_sequence([torch.tensor(x) for x in b], batch_first=True)

mae = F.l1_loss(a.float(), b.float()).item()
mae

1.8571428060531616